# Linear Regression : 설명을 위한 선형회귀분석
 
 1) 종속변수 y : 만성폐쇄성폐질환 (COPD) 삶의 질 평가 검사(CAT) 증상점수
 *  COPD 평가검사(COPD assessment test, CAT)
 * CAT는 8가지 설문 사항을 통해 환자의 삶의 질 즉, 호흡기 증상, 활동 정도, 수면, 자신감을 지표로 삼고 평가
 * 0(삶의 질 가장 좋음)~40점(삶의 질 가장 나쁨)까지의 점수로 나타남 
 
2) 독립변수 x (7개)
 * age :나이, 연속형
 * sex :성별(1:남성, 0:여성), 범주형
 * FEV1%: 1초간 노력성 폐활량(Forced Expiratory Volume in One second, FEV1), 연속형, 
  - (> 70 :mild, 60-69 :moderate ,50-59:moderagely severe, 35-49:severe, <35 :very severe)
 * Smoke_pack_year : 갑년(pack-year), 사람의 담배 노출을 측정하는 데 사용되는 지표, 하루평균 담배소비량(갑)×흡연기간(년)을 의미
 * Chol :  혈중 콜레스트롤 (mg / dl 단위), 연속형
 * Comorbid : 동반질환여부 (1:유, 0:무), 범주형
 * premium : 월평균 건강보험료(소득수준 의미), 연속형
 
 * COPD 환자의 80% 이상에서 10갑년 이상의 흡연력. 을 가지며, 특히 남자에서는 90% 이상 흡연과 관련

### 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [ ]:
df=pd.read_csv('copdcat.csv')

In [ ]:
df.describe()

In [ ]:
X=df.drop(["CATScore"],axis=1)
y=df["CATScore"]

### 그래프를 활용한 탐색적 분석

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.pairplot(df, x_vars=X.columns, y_vars='CATScore', kind='reg', 
             plot_kws={'line_kws':{'color':'red'}, 'scatter_kws': {'alpha': 0.01}})
plt.show()

###  Model1 :기본 모델구축
* Adj. R-squared : 0.74
* 유의수준 5% 하에서 유의하지 않은 변수 : Chol
* FEV1%(폐활량 테스트 점수)가 높을 수록, 연평균 보험료가 높을 수록 만성폐쇄성폐질환자의 삶의 질 증상점수가 낮아짐(삶의 질이 좋아짐)
* 나이가 많을 수록, 성별이 여성대비 남성인 경우, 과거 흡연력이 클수록, 동반질환이 있을 수록 COPD환자의 증상 점수는 높아짐 

In [ ]:
import statsmodels.api as sm
model_reg = sm.OLS(y,sm.add_constant(X))
result_reg = model_reg.fit()
result_reg.summary()

#### Linearity

In [ ]:
sns.scatterplot(x=df['CATScore'], y=result_reg.fittedvalues, alpha=0.1)
plt.plot([0, 30], [0, 30], color='red')
plt.ylabel("Fitted values")
plt.show()

In [ ]:
# linearity
sns.scatterplot(x=result_reg.fittedvalues, y=result_reg.resid, alpha=0.1)
plt.axhline(y=0, color='red')
plt.xlabel("Fitted values")
plt.ylabel("Residual")
plt.show()

#### Normality

In [ ]:
sm.qqplot(result_reg.resid, line='r')
plt.title("Normal Q-Q plot")
plt.show()

#### Equal Variance

In [ ]:
sns.scatterplot(x=result_reg.fittedvalues, y=result_reg.resid, alpha=0.1)
plt.axhline(y=0, color='red')
plt.xlabel("Fitted values")
plt.ylabel("Residual")
plt.show()

### 모형 고급화 방안

In [ ]:
# More on linearity
df1=df.join(pd.DataFrame({'residual': result_reg.resid}))
sns.pairplot(df1, x_vars=X.columns, y_vars='residual', kind='reg', 
             plot_kws={'line_kws':{'color':'red'}, 'scatter_kws': {'alpha': 0.01}})


### Molde2 : X 변수 변환 추가

In [ ]:
X=df.drop(["CATScore"],axis=1)
y=df["CATScore"]
X["Smoke_pack_year2"]= df["Smoke_pack_year"]**2
model_reg2 = sm.OLS(y,sm.add_constant(X))
result_reg2 = model_reg2.fit()
result_reg2.summary()

#### Linearity

In [ ]:
sns.scatterplot(x=df['CATScore'], y=result_reg2.fittedvalues, alpha=0.1)
plt.plot([0, 30], [0, 30], color='red')
plt.ylabel("Fitted values")
plt.show()

#### Normality

In [ ]:
sm.qqplot(result_reg2.resid, line='r')
plt.title("Normal Q-Q plot")
plt.show()

#### Equal variance

In [ ]:
sns.scatterplot(x=result_reg2.fittedvalues, y=result_reg2.resid, alpha=0.1)
plt.axhline(y=0, color='red')
plt.xlabel("Fitted values")
plt.ylabel("Residual")
plt.show()

## 다중공선성 확인
### correlation

In [ ]:
plt.figure(figsize=(10,5))
sns.heatmap(X.corr(),cmap='coolwarm',annot=True)
plt.show()

### VIF 계산

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
X_design = sm.add_constant(X)
vif = pd.DataFrame()
vif["Features"] = X.columns
vif["VIF Factor"] = [np.round(variance_inflation_factor(X_design.values, i+1),4) 
                     for i in range(len(X.columns))] 
vif

## 이상점, 영향점

In [ ]:
result_influ = result_reg2.get_influence().summary_frame() #slow
result_influ

#### Studentized Residual, Leverage Hii

In [ ]:
plt.figure(figsize=(5,5))
sns.scatterplot(data=result_influ, y="student_resid", x="hat_diag", alpha=0.5)
plt.show()

#### Cook's distnace, Dffits

In [ ]:
plt.figure(figsize=(5,5))
sns.scatterplot(data=result_influ, y="dffits", x="cooks_d", alpha=0.5)
plt.show()

#### 영향점 상위 10개

In [ ]:
result_influ_sort = result_influ.sort_values('cooks_d', ascending=False)
result_influ_sort[['cooks_d','student_resid','dffits','hat_diag']].head(10)

####  [해석을 위한 참고] Model2 : 이차항을 포함한 회귀분석
* 선형 회귀 모형의 actual CAT Score 와 predicted CAT Score 그림을 보았을 때, curve의 형태를 보이는 선형으로는 설명 불가능한 부분이 보임
* 탐색적 그래프 분석에서 Smoke_pack_year 변수가 이차항 효과를 가지는 것으로 보여져, 해당 변수의 이차항을 추가
* 후진제거법으로 Chol 과 Smoke_pack_year 변수는 삭제함
* 폐 기능 검사의 노력성 폐활량 FEV1% 점수가 나쁠수록, 흡연력이 길수록, 동반질환이 있을 수록, 연 평균 보험료(소득수준)이 적을수록,
여성보다는 남성이, 나이가 많을 수록  삶의 질 검사(CAT) 증상 점수가 높은 점수를 보인다 (증상 악화, 삶의 질이 나쁨을 뜻함) 